In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

Question 1 (15 points): Read in the data from the given CSV file. Down-select to create a
dataframe that only includes three columns (sport, height, and weight) for all of the males
who competed in either basketball or cycling, then perform dropna() to remove any missing
rows. This should result in a dataframe with 3 columns and 450 rows. Convert the nonnumerical feature (sport) to numerical values. Further, convert the height values from meters
to centimeters. Include in your report a brief description (1-2 sentences) of why it might be
a good idea to use the data in centimeters rather than meters for k-nearest neighbors.

In [11]:
# Read in CSV here
athletes_data = pd.read_csv("all-rio-2016-athletes-excel.csv")
#print(athletes_data.columns)

athletes_data = pd.read_csv("all-rio-2016-athletes-excel.csv")
athletes_data = athletes_data[ (athletes_data['sport'] == "basketball") | (athletes_data['sport'] == "cycling")]
athletes_data = athletes_data.loc[athletes_data["sex"] == "male"]

athletes_data = athletes_data.loc[:, ["height (m)", "weight (kg)", "sport"]]
athletes_data = athletes_data.dropna(axis=0)
athletes_data["height (m)"] *= 100

Index(['name', 'sport', 'nationality', 'sex', 'date_of_birth',
       'age (at start of games)', 'height (m)', 'weight (kg)', 'BMI', 'gold',
       'silver', 'bronze', 'Unnamed: 12', 'Unnamed: 13'],
      dtype='object')


Question 2 (15 points): For each class (basketball, cycling) and feature (height, weight),
compute the following statistics: minimum, maximum, mean, standard deviation, and signalto-noise ratio (SNR = mean/standard deviation). Create a table that shows the statistics
for each class and feature. Do any of the statistics give rise to concern? Provide a brief
description (1-2 sentences) in the report.

Question 3 (5 points): Divide the data into training and test data using the standard 80-20
ratio. Apply 10-fold cross-validation to the training data.

Question 4 (10 points): Create a scatter plot of the training data, where you color the
basketball data and cycling data differently. Make sure to label your axes and include a
legend for the labels. Does the scatter plot indicate potential success or failure with respect
to separating the two classes using KNN? Why? Provide a brief description (1-2 sentences)
in the report.

Question 5 (25 points): Create and iterate over different numbers of neighbors from 1 to 21
(inclusive, but odd numbers only). For each hyperparameter, calculate the mean and standard deviation of classification accuracy across the different folds of the data. Create a table
for your report showing the mean and standard deviation for each hyperparameter. Select
which hyperparameter to use based on the mean classification accuracy on the validation
data. Note which hyperparameter value is selected in the report.


Question 6 (10 points): ): Build a new K-nearest neighbor classifier using all your training
data with the hyperparameter defined in Question 5. Use the resulting model to classify the
test data (which haven’t used until now). Calculate and report overall training and test data
performance.


Question 7 (20 points): Create a 2D plot that visualizes the classifier’s performance (a
decision boundary). Run a mesh of data through the classifier to determine basketball and
cycling decision regions. Color them two different light colors (alpha=0.2) that are easy to
visually separate. Then, overlay the test data using two different colors for basketball and
cycling. Make sure all plots are labeled and that a legend is included.